<a href="https://colab.research.google.com/github/keinam53/Machine_learning/blob/main/Uczenie_nienadzorowane/3_Reguly_asocjacyjne/2_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Apriori cz2
1. [Import bibliotek](#0)
2. [Załadownaie danych](#1)
3. [Przygotowanie danych](#2)
4. [Kodowanie transakcji](#3)
5. [Algorytm Apriori](#4)




### <a name='0'></a> Import bibliotek

In [1]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

### <a name='1'></a> Załadownaie danych

In [2]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2021-08-19 16:09:59--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.167.128, 74.125.133.128, 74.125.140.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.167.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv’

products.csv        100%[===================>]   2.07M  --.-KB/s    in 0.02s   

2021-08-19 16:09:59 (109 MB/s) - ‘products.csv’ saved [2166953/2166953]

--2021-08-19 16:09:59--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 66.102.1.128, 172.253.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet-stream]
Sa

In [3]:
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name'])
products.head()

product_id                                       product_name
0           1                         Chocolate Sandwich Cookies
1           2                                   All-Seasons Salt
2           3               Robust Golden Unsweetened Oolong Tea
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...
4           5                          Green Chile Anytime Sauce

In [4]:
orders = pd.read_csv('orders.csv', usecols=['order_id', 'product_id'])
orders.head()

order_id  product_id
0         1       49302
1         1       11109
2         1       10246
3         1       49683
4         1       43633

### <a name='2'></a> Przygotowanie danych

In [5]:
data = pd.merge(orders, products, how='inner', on='product_id', sort=True)
data = data.sort_values(by='order_id')
data.head()

order_id  product_id                                   product_name
588447          1       22035                    Organic Whole String Cheese
256931          1       10246                          Organic Celery Hearts
277441          1       11109  Organic 4% Milk Fat Whole Milk Cottage Cheese
1194893         1       43633           Lightly Smoked Sardines in Olive Oil
1302365         1       47209                           Organic Hass Avocado

In [6]:
data.describe()

order_id  product_id
count 1384617.00  1384617.00
mean  1706297.62    25556.24
std    989732.65    14121.27
min         1.00        1.00
25%    843370.00    13380.00
50%   1701880.00    25298.00
75%   2568023.00    37940.00
max   3421070.00    49688.00

In [7]:
# rozkład produktów
data['product_name'].value_counts()

Banana                                 18726
Bag of Organic Bananas                 15480
Organic Strawberries                   10894
Organic Baby Spinach                    9784
Large Lemon                             8135
                                       ...  
Cherry Fruit Spread                        1
Dark Magic K Cup Coffee                    1
Red Thai Curry Simmer Sauce                1
Whole Wheat, No. 17 Spaghetti              1
Organic Sea Salt 70% Dark Chocolate        1
Name: product_name, Length: 39123, dtype: int64

In [8]:
# liczba transakcji
data['order_id'].nunique()

131209

In [9]:
transactions = data.groupby(by='order_id')['product_name'].apply(lambda name: ','.join(name))
transactions

order_id
1          Organic Whole String Cheese,Organic Celery Hea...
36         Super Greens Salad,Spring Water,Organic Garnet...
38         Green Peas,Organic Baby Arugula,Shelled Pistac...
96         Organic Raspberries,Organic Pomegranate Kernel...
98         Uncured Applewood Smoked Bacon,Black Beans,Org...
                                 ...                        
3421049    Gluten Free Rice Bread,Organic Baby Broccoli,O...
3421056    Sparkling Lemon Water,Brioche Buns,Homestyle C...
3421058    Classic Britannia Crisps,Club Soda Lower Sodiu...
3421063    Natural Artesian Water,Organic Half & Half,Twi...
3421070    Broccoli Florettes,Organic Unsweetened Almond ...
Name: product_name, Length: 131209, dtype: object

In [10]:
transactions = transactions.str.split(',')
transactions

order_id
1          [Organic Whole String Cheese, Organic Celery H...
36         [Super Greens Salad, Spring Water, Organic Gar...
38         [Green Peas, Organic Baby Arugula, Shelled Pis...
96         [Organic Raspberries, Organic Pomegranate Kern...
98         [Uncured Applewood Smoked Bacon, Black Beans, ...
                                 ...                        
3421049    [Gluten Free Rice Bread, Organic Baby Broccoli...
3421056    [Sparkling Lemon Water, Brioche Buns, Homestyl...
3421058    [Classic Britannia Crisps, Club Soda Lower Sod...
3421063    [Natural Artesian Water, Organic Half & Half, ...
3421070    [Broccoli Florettes, Organic Unsweetened Almon...
Name: product_name, Length: 131209, dtype: object

### <a name='3'></a> Kodowanie transakcji

In [11]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
transactions_encoded = encoder.transform(transactions, sparse=True)
transactions_encoded

<131209x40434 sparse matrix of type '<class 'numpy.bool_'>'
	with 1442410 stored elements in Compressed Sparse Row format>

In [12]:
transactions_encoded_df = pd.DataFrame(transactions_encoded.toarray(), columns=encoder.columns_)

### <a name='4'></a> Algorytm Apriori

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.01, use_colnames=True, n_jobs=-1)
supports = supports.sort_values(by='support', ascending=False)
supports.head(10)

support                  itemsets
8       0.14                  (Banana)
7       0.12  (Bag of Organic Bananas)
76      0.08    (Organic Strawberries)
41      0.07    (Organic Baby Spinach)
31      0.06             (Large Lemon)
37      0.06         (Organic Avocado)
61      0.06    (Organic Hass Avocado)
100     0.05            (Strawberries)
33      0.05                   (Limes)
69      0.04     (Organic Raspberries)

In [ ]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules = rules.sort_values(by='lift', ascending=False)
rules.head(15)

antecedents               consequents  ...  confidence  lift
27                    ( Bag)             (Clementines)  ...        0.79 36.84
26             (Clementines)                    ( Bag)  ...        0.52 36.84
23                   (Limes)             (Large Lemon)  ...        0.26  4.26
22             (Large Lemon)                   (Limes)  ...        0.20  4.26
18    (Organic Strawberries)     (Organic Raspberries)  ...        0.15  3.63
19     (Organic Raspberries)    (Organic Strawberries)  ...        0.30  3.63
31         (Organic Avocado)             (Large Lemon)  ...        0.18  2.94
30             (Large Lemon)         (Organic Avocado)  ...        0.17  2.94
2     (Organic Hass Avocado)  (Bag of Organic Bananas)  ...        0.33  2.81
3   (Bag of Organic Bananas)    (Organic Hass Avocado)  ...        0.16  2.81
17  (Bag of Organic Bananas)     (Organic Raspberries)  ...        0.11  2.72
16     (Organic Raspberries)  (Bag of Organic Bananas)  ...        0.32  2.72
24    (Organic Strawberries)    (Organic Hass Avocado)  ...        0.14  2.54
25    (Organic Hass Avocado)    (Organic Strawberries)  ...        0.21  2.54
28    (Organic Baby Spinach)         (Organic Avocado)  ...        0.14  2.54

[15 rows x 5 columns]